In [12]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from itertools import chain
import os
os.chdir('/home/y0c07y1/rpc_feature_extraction/rpc_model')
import tqdm
import glob
import gcsfs
from utils.utils import *

In [2]:
# files = glob.glob('*.{}'.format('csv'))
# files.sort()
# files

In [13]:
start_date = '2023-02-15'
gcp_directory = 'gs://msc_fair_airflow/rpc_model/'+start_date+'/'
ref_dates = []
for x in range(0, 5):
    ref_dates.append(str(datetime.strptime(start_date, '%Y-%m-%d').date() - timedelta(days=14)*x))
ref_dates.append(str(datetime.strptime(start_date, '%Y-%m-%d').date() + timedelta(days=7)))

gcp_directoty_individual = 'gs://msc_fair_airflow/rpc_model/'+start_date+'/sample_data_individual_feat/'
gcp_directoty_full = 'gs://msc_fair_airflow/rpc_model/'+start_date+'/sample_data_full_feat/'

In [14]:
files = extract_gcs_files(gcp_directoty_individual)
files.sort()

In [16]:
# import pandas as pd
# start_date = '2022-12-19'
# end_date = '2023-01-16'
# ref_dates = pd.date_range(start=pd.to_datetime(start_date), end=pd.to_datetime(end_date), freq='7D')
# ref_dates = [x.strftime('%Y-%m-%d') for x in ref_dates]

In [17]:
for date in ref_dates:
    print('working on date '+date)
    # read header
    file = [x for x in files if x[-14:-4] == date and 'header' in x][0]
    df_feat_all = pd.read_csv(file)
    
    df_feat_all = df_feat_all[(~pd.isnull(df_feat_all['catalog_item_id']))&(~pd.isnull(df_feat_all['seller_id']))]
    df_feat_all['catalog_item_id'] = df_feat_all['catalog_item_id'].astype('int')
    df_feat_all['catalog_item_id'] = df_feat_all['catalog_item_id'].astype('str')
    df_feat_all['seller_id'] = df_feat_all['seller_id'].astype('int')
    df_feat_all['seller_id'] = df_feat_all['seller_id'].astype('str')
    df_feat_all.drop_duplicates(subset = ['adid','is_mobile','source_id'], inplace=True)
    
    # drop off duplicates catalog_item_id, seller_id
    
#     # read targets
#     file = 'sem_l_'+date+'.csv'
#     df_tmp = pd.read_csv(file, index_col=0)
#     df_feat_all = df_tmp.merge(df_feat_all, left_on = 'catalog_item_id', right_on = 'catalog_item_id', how='left')
    
    # read sem coop features
    files_to_read = [x for x in files if x[-14:-4] == date and 'semcoop' in x]
    
    # left join to ensure item coverage is based on target coverage
    for file in files_to_read:
        print('merge sem coop features')
        df_tmp = pd.read_csv(file)
        df_tmp = df_tmp[(~pd.isnull(df_tmp['catalog_item_id']))&(~pd.isnull(df_tmp['seller_id']))]
        df_tmp['catalog_item_id'] = df_tmp['catalog_item_id'].astype('int')
        df_tmp['catalog_item_id'] = df_tmp['catalog_item_id'].astype('str')
        df_tmp['seller_id'] = df_tmp['seller_id'].astype('int')
        df_tmp['seller_id'] = df_tmp['seller_id'].astype('str')
        df_tmp.drop_duplicates(subset = ['adid','is_mobile','source_id'], inplace=True)
        df_feat_all = df_feat_all.merge(df_tmp, on=['catalog_item_id', 'seller_id', 'source_id', 'adid', 'is_mobile'], how='left')
        del df_tmp
        
    # read catalog feature
    files_to_read = [x for x in files if x[-14:-4] == date and 'catalog' in x]
    for file in files_to_read:
        print('merge catalog features')
        df_tmp = pd.read_csv(file)
        df_tmp = df_tmp[(~pd.isnull(df_tmp['catalog_item_id']))&(~pd.isnull(df_tmp['seller_id']))]
        df_tmp['catalog_item_id'] = df_tmp['catalog_item_id'].astype('int')
        df_tmp['catalog_item_id'] = df_tmp['catalog_item_id'].astype('str')
        df_tmp['seller_id'] = df_tmp['seller_id'].astype('int')
        df_tmp['seller_id'] = df_tmp['seller_id'].astype('str')
        df_tmp.drop_duplicates(subset = ['catalog_item_id','seller_id'], inplace=True)
        df_feat_all = df_feat_all.merge(df_tmp, on=['catalog_item_id', 'seller_id'], how='left')
        del df_tmp
        
    # read site features
    files_to_read = [x for x in files if x[-14:-4] == date and 'site' in x]
    for file in files_to_read:
        print('merge site features')
        df_tmp = pd.read_csv(file)
        df_tmp = df_tmp[(~pd.isnull(df_tmp['catalog_item_id']))&(~pd.isnull(df_tmp['seller_id']))]
        df_tmp['catalog_item_id'] = df_tmp['catalog_item_id'].astype('int')
        df_tmp['catalog_item_id'] = df_tmp['catalog_item_id'].astype('str')
        df_tmp['seller_id'] = df_tmp['seller_id'].astype('int')
        df_tmp['seller_id'] = df_tmp['seller_id'].astype('str')
        df_tmp.drop_duplicates(subset = ['catalog_item_id','seller_id'], inplace=True)
        df_feat_all = df_feat_all.merge(df_tmp, on=['catalog_item_id', 'seller_id'], how='left')
        del df_tmp
        
    # read bounce features
    files_to_read = [x for x in files if x[-14:-4] == date and 'bounce' in x]
    for file in files_to_read:
        print('merge bounce features')
        df_tmp = pd.read_csv(file)
        df_tmp = df_tmp[(~pd.isnull(df_tmp['catalog_item_id']))&(~pd.isnull(df_tmp['seller_id']))]
        df_tmp['catalog_item_id'] = df_tmp['catalog_item_id'].astype('int')
        df_tmp['catalog_item_id'] = df_tmp['catalog_item_id'].astype('str')
        df_tmp['seller_id'] = df_tmp['seller_id'].astype('int')
        df_tmp['seller_id'] = df_tmp['seller_id'].astype('str')
        df_tmp.drop_duplicates(subset = ['catalog_item_id','seller_id'], inplace=True)
        df_feat_all = df_feat_all.merge(df_tmp, on=['catalog_item_id', 'seller_id'], how='left')
        del df_tmp

    upload_csv_to_cloud_storage(df_feat_all, gcp_directoty_full+'df_feat_all_coop_item_'+date+'.csv')

working on date 2023-02-15


/opt/conda/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


merge sem coop features
merge sem coop features
merge sem coop features
merge sem coop features
merge sem coop features
merge sem coop features
merge sem coop features
merge sem coop features
merge sem coop features
merge sem coop features
merge catalog features
merge site features
merge site features
merge bounce features
merge bounce features
working on date 2023-02-01
merge sem coop features


KeyboardInterrupt: 